In [1]:
from sklearn.manifold import TSNE
import numpy as np
import os
from os.path import join
from multiprocessing import Pool
from utils import *
sr = 44100
max_length = sr*4 # ignore samples longer than 4 seconds
fixed_length = sr/4 # trim all samples to 250 milliseconds
limit = None # set this to 100 to only load the first 100 samples

In [15]:
files = [f for f in os.listdir()]
for file in files:
    if file.endswith('wav') == False:
        files.remove(file)
samples1 = list()
for file in files:
    audio,fs = ffmpeg_load_audio(file)
    samples1.append(audio)
print(samples1)

176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''
176400
b''

NameError: name 'chunk_size' is not defined

In [14]:
import numpy as np
import subprocess as sp
import os

DEVNULL = open(os.devnull, 'w')
def ffmpeg_load_audio(filename, sr=44100, mono=False, normalize=True, in_type=np.int16, out_type=np.float32):
    channels = 1 if mono else 2
    format_strings = {
        np.float64: 'f64le',
        np.float32: 'f32le',
        np.int16: 's16le',
        np.int32: 's32le',
        np.uint32: 'u32le'
    }
    format_string = format_strings[in_type]
    command = [
        'ffmpeg',
        '-i', filename,
        '-f', format_string,
        '-acodec', 'pcm_' + format_string,
        '-ar', str(sr),
        '-ac', str(channels),
        '-']
    p = sp.Popen(command, stdout=sp.PIPE, stderr=DEVNULL, bufsize=4096)
    bytes_per_sample = np.dtype(in_type).itemsize
    frame_size = bytes_per_sample * channels
    chunk_size = frame_size * sr # read in 1-second chunks
    raw = b''
    with p.stdout as stdout:
        while True:
            data = stdout.read(int(chunk_size/1000))
            if data:
                raw += data
            else:
                break
    print(data)
    audio = np.frombuffer(raw, dtype=in_type).astype(out_type)
    if channels > 1:
        audio = audio.reshape((-1, channels)).transpose()
    if audio.size == 0:
        return audio, sr
    if issubclass(out_type, np.floating):
        if normalize:
            peak = np.abs(audio).max()
            if peak > 0:
                audio /= peak
        elif issubclass(in_type, np.integer):
            audio /= np.iinfo(in_type).max
    return audio, sr

def load_sample(fn, sr=None,
                max_length=None, fixed_length=None, normalize=True):
    if fn == '': # ignore empty filenames
        return None
    audio, _ = ffmpeg_load_audio(fn, sr, mono=True)
    duration = len(audio)
    if duration == 0: # ignore zero-length samples
        return None
    if max_length and duration >= max_length: # ignore long samples
        return None
    if fixed_length:
        audio.resize(fixed_length)
    max_val = np.abs(audio).max()
    if max_val == 0: # ignore completely silent sounds
        return None
    if normalize:
        audio /= max_val
    return (fn, audio, duration)

In [ ]:
def job(fn):
    return load_sample(fn, sr=sr,
                       max_length=max_length, fixed_length=fixed_length)
pool = Pool()
%time results = pool.map(job, files[:limit])
print ('Processed', len(results), 'samples')
print(results)

In [ ]:
filenames = [x[0] for x in results]
samples = [x[1] for x in results]
durations = [x[2] for x in results]
samples = np.asarray(samples)
np.savetxt(join('filenames.txt'), filenames, fmt='%s')
np.savetxt(join('durations.txt'), durations, fmt='%i')
%time np.save(join('samples.npy'), samples)
print ('Saved', len(valid), 'samples')

In [ ]:
import librosa
from tqdm import *
from skimage.measure import block_reduce

n_fft = 2048
hop_length = n_fft/4
use_logamp = False # boost the brightness of quiet sounds
reduce_rows = 10 # how many frequency bands to average into one
reduce_cols = 1 # how many time steps to average into one
crop_rows = 32 # limit how many frequency bands to use
crop_cols = 32 # limit how many time steps to use
limit = None # set this to 100 to only process 100 samples
window = np.hanning(n_fft)
def job(y):
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=window)
    amp = np.abs(S)
    if reduce_rows > 1 or reduce_cols > 1:
        amp = block_reduce(amp, (reduce_rows, reduce_cols), func=np.mean)
    if amp.shape[1] < crop_cols:
        amp = np.pad(amp, ((0, 0), (0, crop_cols-amp.shape[1])), 'constant')
    amp = amp[:crop_rows, :crop_cols]
    if use_logamp:
        amp = librosa.logamplitude(amp**2)
    amp -= amp.min()
    if amp.max() > 0:
        amp /= amp.max()
    amp = np.flipud(amp) # for visualization, put low frequencies on bottom
    return amp
pool = Pool()
%time fingerprints = pool.map(job, samples[:limit])
fingerprints = np.asarray(fingerprints).astype(np.float32)

In [ ]:
def list_all_files(directory, extensions=None):
    for root, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            base, ext = os.path.splitext(filename)
            joined = os.path.join(root, filename)
            if extensions is None or ext.lower() in extensions:
                yield joined
files = list(list_all_files(join(data_root),['.wav']))
len(files)
print(files)